# 제주 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote
import unicodedata
import re
from tqdm.notebook import tqdm

In [2]:
chromedriver = '/Workspace/02. DataAnalysis/05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

## 1. 인스타 접속하여 로그인

In [3]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [7]:
with open('password.txt') as f:
    password = f.read(8)

In [33]:
email = 'jinyoun9.kim@gmail.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [34]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [36]:
try :
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except :
    pass

In [37]:
try :
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except :
    pass

## 2. 관악구맛집으로 검색

In [39]:
keyword = '관악구맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EA%B4%80%EC%95%85%EA%B5%AC%EB%A7%9B%EC%A7%91'

In [40]:
driver.get(url)
time.sleep(8)

### 3. 첫번째 게시글 열기

In [44]:
driver.find_element_by_css_selector('._9AhH0').click()
time.sleep(2)

### 4. 첫번째 게시글 정보 가져오기

In [45]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [52]:
try:
    content = driver.find_element_by_css_selector('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'20210316 신림동 만성찬팅\n\n중국판 김밥천국이라고나 할까요?\n중국요리가 가격이 9천원을 넘지않고\n식사는 4천원으로 통일, 마라라면은 3천5백원이라니\n\n분위기 또한 아주 좋습니다\n마치 중국에 온 느낌이랄까요?\n밝고 깨끗한게 중국식당 특유의 어두움과\n더러운 환경이 전혀 아니었어요\n교포 사장님 내외 및 직원들이 아주 친절한 것도 그렇구요\n\n음식 맛 자체는 크게 높은 레벨은 아니지만\n가격 생각하면 미친듯한 가성비입니다\n게다가 양도 많아서 아주 놀랐네요\n\n즉 여긴 친구들과 가볍게 맥주한잔하기 딱 좋은 가게입니다\n전 무조건 재방문의사 있습니다\n\n#관악구재영 #관악구맛집\n#신림동재영 #신림동맛집\n#만성찬팅재영 #만성찬팅\n#중국요리재영 #중국요리맛집\n#중식재영 #중식맛집\n\n#wine #酒 #日本酒 #맛스타그램 #먹스타그램 #술스타그램 #instafood #foodie #food #foodstagram #와인 #ワイン'

In [54]:
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#관악구재영',
 '#관악구맛집',
 '#신림동재영',
 '#신림동맛집',
 '#만성찬팅재영',
 '#만성찬팅',
 '#중국요리재영',
 '#중국요리맛집',
 '#중식재영',
 '#중식맛집',
 '#wine',
 '#酒',
 '#日本酒',
 '#맛스타그램',
 '#먹스타그램',
 '#술스타그램',
 '#instafood',
 '#foodie',
 '#food',
 '#foodstagram',
 '#와인',
 '#ワイン']

In [59]:
date = soup.select_one('.FH9sR.Nzb55')['datetime'][:10]
date

'2021-03-21'

In [65]:
try:
    like = soup.select_one('.Nm9Fw').text[4:-1]
except:
    like = 0
like

'1,729'

In [67]:
try:
    place = soup.select_one('.O4GlU').text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'신림역 어딘가'

In [68]:
row = [content, date, like, place, tags]
row

['20210316 신림동 만성찬팅\n\n중국판 김밥천국이라고나 할까요?\n중국요리가 가격이 9천원을 넘지않고\n식사는 4천원으로 통일, 마라라면은 3천5백원이라니\n\n분위기 또한 아주 좋습니다\n마치 중국에 온 느낌이랄까요?\n밝고 깨끗한게 중국식당 특유의 어두움과\n더러운 환경이 전혀 아니었어요\n교포 사장님 내외 및 직원들이 아주 친절한 것도 그렇구요\n\n음식 맛 자체는 크게 높은 레벨은 아니지만\n가격 생각하면 미친듯한 가성비입니다\n게다가 양도 많아서 아주 놀랐네요\n\n즉 여긴 친구들과 가볍게 맥주한잔하기 딱 좋은 가게입니다\n전 무조건 재방문의사 있습니다\n\n#관악구재영 #관악구맛집\n#신림동재영 #신림동맛집\n#만성찬팅재영 #만성찬팅\n#중국요리재영 #중국요리맛집\n#중식재영 #중식맛집\n\n#wine #酒 #日本酒 #맛스타그램 #먹스타그램 #술스타그램 #instafood #foodie #food #foodstagram #와인 #ワイン',
 '2021-03-21',
 '1,729',
 '신림역 어딘가',
 ['#관악구재영',
  '#관악구맛집',
  '#신림동재영',
  '#신림동맛집',
  '#만성찬팅재영',
  '#만성찬팅',
  '#중국요리재영',
  '#중국요리맛집',
  '#중식재영',
  '#중식맛집',
  '#wine',
  '#酒',
  '#日本酒',
  '#맛스타그램',
  '#먹스타그램',
  '#술스타그램',
  '#instafood',
  '#foodie',
  '#food',
  '#foodstagram',
  '#와인',
  '#ワイン']]

### 5. 다음 게시글로 이동

In [71]:
driver.find_element_by_css_selector('._65Bje').click()
time.sleep(2)

In [72]:
# 6을 하기 위해 popup 화면 끄기
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

In [101]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html=parser')

    try:
        content = driver.find_element_by_css_selector('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    date = soup.select_one('.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('.Nm9Fw').text[4:-1]
    except:
        like = 0
    
    try:
        place = soup.select_one('.O4GlU').text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    tags = re.findall(r'#[^\s#,\\]+', content)

    return [content, date, like, place, tags]

### 7. 관악구맛집, 샤로수길맛집

In [79]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/button"}
  (Session info: chrome=92.0.4515.159)


In [82]:
keyword = '관악구맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(1)

In [83]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [90]:
results = []

In [91]:
target = 2

In [98]:
for i in range(target):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('._65Bje').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('._65Bje').click()
        time.sleep(3)
print(results[:2])

[]


In [99]:
results

[]

In [85]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags


In [86]:
df.tail()

,content,date,like,place,tags


In [ ]:
df.to_csv('data/관악구맛집.csv', index=False)

In [ ]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [ ]:
driver.close()

## 2가지 csv 파일의 중복을 제거한 후 통합 

In [ ]:
df.drop_duplicates(subset=['content'], inplace=True)
df.to_csv('관악구핫플레이스.csv', index=False)

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
df = pd.read_csv('관악구핫플레이스.csv')
df.head()

# 해시태그 데이터로 워드클라우드 만들기

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null
# 런타임 다시 시작

In [ ]:
raw_df = pd.read_csv('관악구핫플레이스.csv')
raw_df.tags[:3]

In [ ]:
tags = raw_df.tags[0]
tags

In [ ]:
tags[2:-2].split("', '")[:5]

In [ ]:
a = ['#시작']
a.extend(tags[2:-2].split("', '")[:5])
a

In [ ]:
tags_total = []
for tags in raw_df.tags:
    tags_total.extend(tags[2:-2].split("', '"))

### 1. 해시태그 출현 빈도
#### * collection 모듈

In [ ]:
from collections import Counter, OrderedDict

In [ ]:
s = 'A quick brown'
Counter(s)

In [ ]:
sl = ['A', 'quick', 'brown', 'brown']
Counter(sl)

In [ ]:
d = dict()
d['x'] = 1
d['k'] = 2
d['m'] = 3
d

In [ ]:
od = OrderedDict()
od['x'] = 1
od['k'] = 2
od['m'] = 3
od

In [ ]:
for k, v in od.items():
    print(k, v)

### *해시태그 빈도 집계

In [ ]:
tag_counts = Counter(tags_total)

In [ ]:
# Top 50
tag_counts.most_common(50)

In [ ]:
# 데이터 정제하기
stopwords = ['#jeju','#일상','','#제주','#제주도','#선팔','#제주눈썹문신',
             '#서귀포눈썹문신', '#반영구', '#눈썹문신', '#제주자연눈썹',
             '#제주속눈썹', '#서귀포남자눈썹문신', '#서귀포자연눈썹', '#제주남자눈썹문신', 
             '#서귀포속눈썹', '#서귀포반영구']
'''tag_total_refined = []
for tag in tags_total:
    if tag not in stopwords:
        tag_total_refined.append(tag)'''
tags_total = [tag for tag in tags_total if tag not in stopwords]

tag_counts = Counter(tags_total)
tag_counts.most_common(50)

## 2. 워드 클라우드 만들기

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
! ls -l /usr/share/fonts/truetype/nanu

In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
wordcloud = WordCloud(font_path=path,
                      background_color='white',
                      max_words=100,
                      relative_scaling=0.3,
                      width=800, height=400
                      ).generate_from_frequencies(tag_counts)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()